# Introduction to TFX 
---

In this notebook we will present a high-level overview of TFX, how we are using it for the semantic autotagging model, and the advantages and disadvantages of this way of writing ML pipelines. 

Outline:

    A. Introduction and Background 
        * What is the problem? 
        * What is TFX and how does it help address the problem? 
        * Background on Semantic Autotagging Model 
    B. Demo 
        * Demonstrate TFX pipeline components
        * Demonstrate TFX libraries 
        * Train for a few steps, push model and run inference on raw examples using saved_model_cli
    C. Takeaways and Conclusion
        * Advantages and when to use TFX
        * Disadvantages and when to not use TFX

---
---

## **A. Introduction and Background**
---

#### **A.1 What is the Problem?**

As a data scientist, I am most interested in cool new model architectures and training algorithms. After all, this is the fun part, right? 

In reality, **only a small fraction of real-world ML systems is composed of the ML code**. Getting an ML model to actually work in the real world often requires vast and complex surroudning infrastructure. This is because there are ML-specific risk factors that should be accounted for in system design unless you want to incur **massive ongoing maintenance costs**. What many organizations find is that **development and deployment of ML systems is fast and cheap, but maintenance is difficult and expensive.** Therefore, it is recommended that before focusing on cutting-edge ML modelling techniques, product leaders should invest more time in adopting interoperable ML platforms for their organization. If this investment isn't made, some system-wide anti-patterns are likely to emerge, such as massive amounts of bespoke glue code or pipeline jungles. 



![image](Hidden_technical_debt.jpg)


Developing and deploying AI models is a complex, iterative process. While the freedom to experiment is vital in any data science workflow, engineering precision is required to get a stable model out into production.
Successful AI automation revolves around addressing these often competing needs: efficient, reproducible, and transparent experimentation allowing for rapid R&D iterations, and rigor in deploying and monitoring models.

So how do we address all these concerns? What we really need is an end-to-end ML platform which can help with **all** aspects of the ML lifecycle in order to accelerate ML adoption and make its use durable and sustainable. 

Here is a wish-list of the things we would want from our platform:
 - Agility to accelerate the journey from model ideation to deployment and business impact 
 - Automation to free up data scientists to focus on new models, not maintaing existing models
 - Standardization, consistency, and repeatability
 - Visibility by having a centralized place to collect, visualize, and compare information about data, models, and experiments
 - Simpler processes to update existing models
 - Performance and scalability 
 - Prevention of bugs such as not updating preprocessing steps when a new model is trained 
 - Artifact lineage for a useful paper trail
 
Our platform should allow us to create training pipelines that start with ingestion of new training data and include steps such as data validation, preprocessing, model training, model analysys, and deployment of the model. 

---

#### **A.2 What is TFX and how can it help?**

"TFX is an end-to-end platform for deploying production ML pipelines." 

A TFX pipeline is a sequence of components that implement an ML pipeline which is specifically designed for **scalable, high-performance** machine learning tasks (including modeling, training, serving inference, data validation). Pipeline components are built using **TFX libraries**, which can also be used in isolation.

**Libraries**

 - Tensorflow Data Valdation: Scalable data analysis, powerful visualizations, and ability to validate new data
 - Tensorflow Transform: Scalable feature engineering; eliminate one source of training-serving skew by ensuring preprocessing is the **same** between training and serving
 - Tensorflow Model Analysis: Compute and visualize detailed evaluation metrics for models, especially on **slices** of data
 - Tensorflow Serving: Support for model versioning, multiple models. 
 - ML Metadata: Store and retrieve metadata associated with ML developer and data scientist workflow. "Artifacts are a first class citizen in ML, on par with processes that produce and consume them." 


    
**Pipeline** 

![image](tfx_pipelines.png)

What we want is an approprite balance between flexiblity and standardization. TFX pipelines allow for standardized orchestration and abstraction of glue code between tasks. Furthermore, TFX forces best practice by employing several validation checks at different stages of the pipeline. These checks can ensure that models are trained on high-quality data (data validation), are at least as good or better than current production model (model validation), and are compatible with the deployment environment (serving infrastructure validation). 


---
#### **A.3 Background on Semantic Autotagging Model**

We decided to use TFX for productionization of Semantic Autotagging Model for Germany. 


---
---
## **B. Demo**

TFX is massive, so we will not be able demo either all of capabilities of the libaries or even a complete pipeline. 

In this demo we will create a TFX pipeline skeleton to demonstrate a few of the components. In a production deployment of TFX we would use an orchestrator such as Apache Airflow, Kubeflow Pipelines, or Apache Beam. In an interactive notebook the notebook itself is the orchestrator, running each TFX component as the notebook cells are executed. By running the pipeline interactively we can easily visualize some of the features of TFX. We will be able to, for example, inspect the inputs to and outputs from each component, investigate how data is being written to the metadata store, and demystify some what is happening under the hood in the TFX components. 

In [1]:
import os
import pprint
import tempfile
import urllib

import pandas as pd
from sklearn.model_selection import train_test_split

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components.example_gen.custom_executors import parquet_executor 
from tfx.components.example_gen.component import FileBasedExampleGen

from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2

from tensorflow_metadata.proto.v0 import schema_pb2 
from tensorflow_metadata.proto.v0 import statistics_pb2
from tensorflow_metadata.proto.v0 import anomalies_pb2

from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2

import tensorflow_data_validation as tfdv 
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

/home/jupyter/venv/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [2]:
class ReadonlyMetadataStore(object):
    """An ml-metadata store that provides read-only methods for notebooks."""

    def __init__(self, store, base_dir):
        """Initializes a ReadonlyMetadataStore with given store.
        Args:
          store: An instance of `metadata_store.MetadataStore`.
        """
        self.metadata_store = store
        self.base_dir = base_dir
        
    def display_types(self):
        # Helper function to render dataframes for the artifact and execution types
        table = {'id': [], 'name': []}
        types = self.metadata_store.get_artifact_types()
        for a_type in types:
            table['id'].append(a_type.id)
            table['name'].append(a_type.name)
        return pd.DataFrame(data=table)
    
    def display_artifacts(self, artifacts):
        # Helper function to render dataframes for the input artifacts
        table = {'artifact id': [], 'type': [], 'uri': []}
        for a in artifacts:
            table['artifact id'].append(a.id)
            artifact_type = self.metadata_store.get_artifact_types_by_id([a.type_id])[0]
            table['type'].append(artifact_type.name)
            table['uri'].append(a.uri.replace(self.base_dir, './'))
        return pd.DataFrame(data=table)
    
    @staticmethod
    def display_properties(node):
        # Helper function to render dataframes for artifact and execution properties
        table = {'property': [], 'value': []}
        for k, v in node.properties.items():
            table['property'].append(k)
            table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
        for k, v in node.custom_properties.items():
            table['property'].append(k)
            table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
        return pd.DataFrame(data=table)

    def get_df_from_single_artifact_or_execution(self, obj):
        """Returns a `pd.DataFrame` based on an artifact/execution properties.
        Args:
          obj: An instance of `metadata_store_pb2.Artifact` or
               `metadata_store_pb2.Execution`.
        Returns:
          A `pd.DataFrame` to display the properties of an artifact/execution.
        """
        data = {}
        if isinstance(obj, metadata_store_pb2.Artifact):
            data['URI'] = obj.uri
        for p in obj.properties:
            data[p.upper()] = _get_value_str(obj.properties[p])
        for p in obj.custom_properties:
            data[p.upper()] = _get_value_str(obj.custom_properties[p])
        return pd.DataFrame.from_dict(
            data=data, orient='index', columns=['']).fillna('-')

    def get_df_from_artifacts_or_executions(self, objects):
        """Returns a `pd.DataFrame` of given artifacts'/executions' properties."""
        data = {}
        for obj in objects:
            col_map = {}
            if isinstance(obj, metadata_store_pb2.Artifact):
                col_map['URI'] = obj.uri
            for p in obj.properties:
                col_map[p.upper()] = _get_value_str(obj.properties[p])
            for p in obj.custom_properties:
                col_map[p.upper()] = _get_value_str(obj.custom_properties[p])
            data[obj.id] = col_map
        df = pd.DataFrame.from_dict(data=data, orient='index').fillna('-')
        df.index.name = 'ID'
        return df

    def get_artifact_df(self, artifact_id):
        """Returns a `pd.DataFrame` for an artifact with `artifact_id`.
        Args:
          artifact_id: An `int` indicating the id of an artifact in the store.
        Returns:
          A `pd.DataFrame` to display the properties of the artifact corresponding
          to `artifact_id` or None if no such artifact exists in the store.
        """
        artifacts = self.metadata_store.get_artifacts_by_id([artifact_id])
        return (
            self.get_df_from_single_artifact_or_execution(artifacts[0])
            if artifacts else None
        )

    def get_execution_df(self, execution_id):
        """Returns a `pd.DataFrame` for an execution with `execution_id`.
        Args:
          execution_id: An `int` indicating the id of an execution in the store.
        Returns:
          A `pd.DataFrame` to display the properties of the execution corresponding
          to `execution_id` or None if no such execution exists in the store.
        """
        executions = self.metadata_store.get_executions_by_id([execution_id])
        return (
            self.get_df_from_single_artifact_or_execution(executions[0])
            if executions else None
        )

    def get_artifacts_of_type_df(self, type_name):
        """Returns a `pd.DataFrame` for all artifacts of given `type_name`.
        Args:
          type_name: A `str` indicating the name of an artifact type in the store.
        Returns:
          A `pd.DataFrame` to display the properties of all artifacts with given
          type in the store.
        """
        return self.get_df_from_artifacts_or_executions(
            self.metadata_store.get_artifacts_by_type(type_name))

    def get_executions_of_type_df(self, type_name):
        """Returns a `pd.DataFrame` for all executions of given `type_name`.
        Args:
          type_name: A `str` indicating the name of an execution type in the store.
        Returns:
          A `pd.DataFrame` to display the properties of all executions with given
          type in the store.
        """
        return self.get_df_from_artifacts_or_executions(
            self.metadata_store.get_executions_by_type(type_name))

    def get_source_artifact_of_type(self, artifact_id, source_type_name):
        """Returns the source artifact of `source_type_name` for `artifact_id`.
        This method recursively traverses the events and associated executions that
        led to generating `artifact_id` to find an artifact of type
        `source_type_name` that was an input for these events.
        Args:
          artifact_id: A `int` indicating the id of an artifact.
          source_type_name: A `str` indicating the type of an artifact that is
              a direct or indirect input for generating `artifact_id`.
        Returns:
          A `metadata_store_pb2.Artifact` of type `source_type_name` that is a
          direct/indirect input for generating `artifact_id` or `None` if no such
          artifact exists.
        """
        a_events = self.metadata_store.get_events_by_artifact_ids([artifact_id])
        for a_event in a_events:
            if _is_input_event(a_event):
                continue
            [execution] = self.metadata_store.get_executions_by_id(
              [a_event.execution_id])
            e_events = self.metadata_store.get_events_by_execution_ids([execution.id])
            for e_event in e_events:
                if _is_output_event(e_event):
                    continue
            [artifact] = self.metadata_store.get_artifacts_by_id(
                [e_event.artifact_id])
            [artifact_type] = self.metadata_store.get_artifact_types_by_id(
                [artifact.type_id])
            if artifact_type.name == source_type_name:
                return artifact
            input_artifact = self.get_source_artifact_of_type(
                artifact.id, source_type_name)
            if input_artifact:
                return input_artifact

    def get_dest_artifact_of_type(self, artifact_id, dest_type_name):
        """Returns the destination artifact of `dest_type_name` from `artifact_id`.
        This method recursively traverses the events and associated executions that
        consumed `artifact_id` to find an artifact of type `dest_type_name` that was
        an output for these events.
        Args:
          artifact_id: A `int` indicating the id of an artifact.
          dest_type_name: A `str` indicating the type of an artifact that is
              a output of an event that directly/indirectly consumed `artifact_id`.
        Returns:
          A `metadata_store_pb2.Artifact` of type `dest_type_name` that is a
          direct/indirect output from `artifact_id` or `None` if no such artifact
          exists.
        """
        a_events = self.metadata_store.get_events_by_artifact_ids([artifact_id])
        for a_event in a_events:
            if _is_output_event(a_event):
                continue
            [execution] = self.metadata_store.get_executions_by_id(
              [a_event.execution_id])
            e_events = self.metadata_store.get_events_by_execution_ids(
              [execution.id])
            for e_event in e_events:
                if _is_input_event(e_event):
                    continue
                [artifact] = self.metadata_store.get_artifacts_by_id(
                [e_event.artifact_id])
                [artifact_type] = self.metadata_store.get_artifact_types_by_id(
                [artifact.type_id])
                if artifact_type.name == dest_type_name:
                    return artifact
                dest_artifact = self.get_dest_artifact_of_type(
                    artifact.id, dest_type_name)
            if dest_artifact:
                return dest_artifact

    def get_execution_for_output_artifact(self, artifact_id, type_name):
        """Returns the execution of `type_name` that generated `artifact_id`.
        Args:
          artifact_id: A `int` indicating the id of an artifact.
          type_name: A `str` indicating the type of an Execution that generated
            `artifact_id`.
        Returns:
          A `metadata_store_pb2.Execution` of type `type_name` that generated
          `artifact_id` or `None` if no such execution exists.
        """
        a_events = self.metadata_store.get_events_by_artifact_ids([artifact_id])
        for a_event in a_events:
            if _is_input_event(a_event):
                continue
            [execution] = self.metadata_store.get_executions_by_id(
              [a_event.execution_id])
            [execution_type] = self.metadata_store.get_execution_types_by_id(
              [execution.type_id])
            if execution_type.name == type_name:
                return execution


def _is_input_event(event):
    return event.type in [metadata_store_pb2.Event.DECLARED_INPUT, 
                          metadata_store_pb2.Event.INPUT]

def _get_value_str(p):
    """Returns a string representation of a `metadata_store_pb2.Value` object."""
    if p.int_value:
        return str(p.int_value)
    if p.string_value:
        return p.string_value
    if p.double_value:
        return str(p.double_value)
    return ''

def _is_output_event(event):
    return event.type == metadata_store_pb2.Event.OUTPUT 

class TFXArtifactTypes(object):
    """Constants for different TFX artifact type names."""
    EXAMPLES = 'Examples'
    SCHEMA = 'Schema'
    EXAMPLE_STATS = 'ExampleStatistics'
    EXAMPLE_VALIDATION = 'ExampleAnomalies'
    TRANSFORMED_EXAMPLES = 'TransformGraph'
    MODEL = 'Model'
    MODEL_EVAL = 'ModelEvaluation'


class TFXExecutionTypes(object):
    """Constants for different TFX execution type names."""
    EXAMPLE_GEN = 'tfx.components.example_gen.component.FileBasedExampleGen'
    STATISTICS_GEN = 'tfx.components.statistics_gen.component.StatisticsGen'
    SCHEMA_GEN = 'tfx.components.schema_gen.component.SchemaGen'
    EXAMPLE_VALIDATION = 'tfx.components.example_validator.component.ExampleValidator'
    TRANSFORM = 'tfx.components.transform.component.Transform'
    TRAINER = 'tfx.components.trainer.component.Trainer'
    EVALUATOR = 'tfx.components.evaluator.component.Evaluator'
    PUSHER = 'tfx.components.pusher.component.Pusher'

#### **Define Data Paths and Setup the Interactive Context**

We will be using a small sample of the data that we are actually using to train and evaluate the semantic autotagging model. 

In [4]:
_data_root_test = "gs://ml-sandbox-tagging-tfx-experiments/testdata/parquet/"
_train_sample_path = os.path.join(_data_root_test, "train.parquet")
_eval_sample_path = os.path.join(_data_root_test, "eval.parquet")

Here we can print out the data to get a feel for what we'll be working with. 

The goal is to predict appropriate tags from raw text descriptions of the video content. 

In [5]:
train_df = pd.read_parquet(_train_sample_path)
display(train_df)

,features,series_ep_tags
0,Ein amerikanischer Militärbeamter untersucht d...,"[20th century, thriller, the middle east, myst..."
1,Aufschlussreiche Dokumentation über den Weltme...,"[sport skills & tricks, bio, mental health, ad..."
2,"Ein Söldnerteam wird von der CIA beauftragt, n...","[thriller, military personnel, soldier, action..."
3,Ein Amerikaner lässt sich in einem Dorf im feu...,"[the far east, thriller, china, 19th century, ..."
4,Ester und ihre Mutter führen ein scheinbar bes...,"[criminal, usa, latin america, violent, thrill..."
5,"Ein Filmstar bereitet sich darauf vor, zum vie...","[funny, dysfunctional, comedy, rivalry, romant..."
6,Sa 07/03: Es wurden große Anstrengungen untern...,"[medical, health, the far east, china, busines..."
7,"Ein legendärer, aber pensionierter Killer wird...","[assassin, rivers & lakes, rural, thriller, my..."
8,Zu Ehren ihres verstorbenen Freundes begeben s...,"[usa, funny, dysfunctional, bromance, comedy, ..."
9,Dokumentarfilm über ein turbulentes Jahr im ge...,"[music, classical music, crime, performing art..."


In [6]:
_pipeline_root = tempfile.mkdtemp(prefix='pipeline')
_pipeline_name = 'demo_pipeline'

context = InteractiveContext(pipeline_name=_pipeline_name, pipeline_root=_pipeline_root)

In [7]:
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = '/tmp/pipelinerlw8w3yc/metadata.sqlite' # REPLACE with new location
connection_config.sqlite.connection_mode = 3 #READWRITE_OPENCREATE
store = metadata_store.MetadataStore(connection_config)
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

read_only_store = ReadonlyMetadataStore(store, base_dir)

#### **Example Gen**

The first step in our pipeline is the ExampleGen component, which is used to ingest data from external sources. This component can take pre-split data or split the data into training and evaluation sets. We can take in data from different sources, including from CSV files and directly from BigQuery. 

Here we will be taking in data directly from parquet files in GCS. 

In [8]:
input_config = example_gen_pb2.Input(splits=[
                    example_gen_pb2.Input.Split(name='train', pattern='train*'),
                    example_gen_pb2.Input.Split(name='eval', pattern='eval*')
])

example_gen = FileBasedExampleGen(input_base=_data_root_test, 
                                  custom_executor_spec=executor_spec.ExecutorClassSpec(parquet_executor.Executor),
                                  input_config=input_config)

context.run(example_gen)

ExecutionResult(
    component_id: FileBasedExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/pipelinerlw8w3yc/FileBasedExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:7227,xor_checksum:1603792024,sum_checksum:1603792024\nsplit:eval,num_files:1,total_bytes:8203,xor_checksum:1603792024,sum_checksum:1603792024"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

In [9]:
display(read_only_store.get_artifacts_of_type_df(TFXArtifactTypes.EXAMPLES))
display(read_only_store.get_executions_of_type_df(TFXExecutionTypes.EXAMPLE_GEN))

,URI,SPLIT_NAMES,SPAN,PAYLOAD_FORMAT,INPUT_FINGERPRINT,STATE
ID,,,,,,
1,/tmp/pipelinerlw8w3yc/FileBasedExampleGen/exam...,"[""train"", ""eval""]",0,FORMAT_TF_EXAMPLE,"split:train,num_files:1,total_bytes:7227,xor_c...",published


,OUTPUT_DATA_FORMAT,OUTPUT_CONFIG,CUSTOM_CONFIG,STATE,SPAN,INPUT_BASE,PIPELINE_ROOT,RUN_ID,INPUT_FINGERPRINT,INPUT_CONFIG,PIPELINE_NAME,COMPONENT_ID,VERSION
ID,,,,,,,,,,,,,
1,6,{},None,complete,0,gs://ml-sandbox-tagging-tfx-experiments/testda...,/tmp/pipelinerlw8w3yc,2020-11-03T13:19:29.957753,"split:train,num_files:1,total_bytes:7227,xor_c...","{\n ""splits"": [\n {\n ""name"": ""train""...",demo_pipeline,FileBasedExampleGen,None


In [10]:
def make_dataset(examples, split):
    """
    Create a dataset from examples artifact
    """
    uri = os.path.join(examples.get()[0].uri, split)
    
    tfrecord_filenames = [os.path.join(uri, name) for name in os.listdir(uri)]
    dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
    
    return dataset

We can create datasets from the generated examples and display them. 

In [11]:
train_dataset = make_dataset(example_gen.outputs['examples'], 'train')
eval_dataset = make_dataset(example_gen.outputs['examples'], 'eval')

print('{:=^80}'.format(' Example '))

# Iterate over the first record and decode it.
for tfrecord in train_dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

=================================== Example ====================================
features {
  feature {
    key: "features"
    value {
      bytes_list {
        value: "Ein amerikanischer Milit\303\244rbeamter untersucht den Tod einer Hubschrauberpilotin im Golfkrieg von 1990, um zu entscheiden, ob ihre Handlungen eine posthume Ehrenmedaille verdienen. In der Zwischenzeit versucht er, sich mit einem tragischen Vorfall auseinanderzusetzen, den er w\303\244hrend des Konflikts selbst verursacht und vertuscht hat. Drama mit Denzel Washington, Meg Ryan, Lou Diamond Phillips, Scott Glenn und Matt Damon. .. Krieg Theater. Courage Under Fire  "
      }
    }
  }
  feature {
    key: "series_ep_tags"
    value {
      bytes_list {
        value: "20th century"
        value: "thriller"
        value: "the middle east"
        value: "mystery"
        value: "gulf war"
        value: "war"
        value: "1990s"
        value: "ethics & morality"
        value: "military personnel"
        val

---
#### **Statistics Gen**

StatisticsGen will generate feature statistics over the data. This will allow us to compare the statistical properties of our training and evaluation data.

In [12]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/pipelinerlw8w3yc/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [13]:
display(read_only_store.get_artifacts_of_type_df(TFXArtifactTypes.EXAMPLE_STATS))
display(read_only_store.get_executions_of_type_df(TFXExecutionTypes.STATISTICS_GEN))

,URI,SPLIT_NAMES,PRODUCER_COMPONENT,NAME,STATE
ID,,,,,
2,/tmp/pipelinerlw8w3yc/StatisticsGen/statistics/2,"[""train"", ""eval""]",StatisticsGen,statistics,published


,EXCLUDE_SPLITS,COMPONENT_ID,STATE,PIPELINE_ROOT,PIPELINE_NAME,RUN_ID,STATS_OPTIONS_JSON
ID,,,,,,,
2,[],StatisticsGen,complete,/tmp/pipelinerlw8w3yc,demo_pipeline,2020-11-03T13:19:35.283741,None


**Using TFDV library**. We can also use the tensorflow data validation directly. Here is one example of how we can use it to visualize dataset statistics. 

TFDV can easily be used outside of the context of a TFX pipeline for IDA and EDA.

In [14]:
stats_path = statistics_gen.outputs['statistics'].get()[0].uri
train_statistics_path = os.path.join(stats_path, 'train/stats_tfrecord')
eval_statistics_path = os.path.join(stats_path, 'eval/stats_tfrecord')

train_statistics = tfdv.load_statistics(train_statistics_path)
eval_statistics = tfdv.load_statistics(eval_statistics_path)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [15]:
tfdv.visualize_statistics(lhs_statistics=train_statistics,
                          rhs_statistics=eval_statistics)

---

#### **Schema**

We can use a schema to codify our expectations about the data we're working with. The SchemaGen component will automatically create a schema, but it typically requires further manual curation. 

**Using TFDV library**

In [16]:
schema = tfdv.infer_schema(train_statistics, infer_feature_shape=True)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'features',STRING,required,,'features'
'series_ep_tags',STRING,required,"[1,inf)",'series_ep_tags'


/home/jupyter/venv/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:138: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'features',"'Aufschlussreiche Dokumentation über den Weltmeister-Surfer Andy Irons und seine Kämpfe mit bipolaren Störungen und Sucht. Starke Sprache / reife Themen / blinkende Bilder. (2018) (100 Minuten). .. Dokumentarfilm. Andy Irons: Kissed By God ', 'Dokumentarfilm über ein turbulentes Jahr im gefeierten Bolschoi-Ballett in Moskau nach einem bösartigen Angriff auf seinen künstlerischen Leiter. Starke Sprache / Gewalt / Sex / blinkende Bilder. (2015) (86 Minuten). .. Dokumentarfilm. Bolshoi Babylon ', 'Ein Amerikaner lässt sich in einem Dorf im feudalen China nieder und stellt Waffen für verschiedene feudale Clans von Kriegern her. Wenn eine Goldsendung in die Stadt gebracht wird, bricht der Konflikt zwischen den Fraktionen in Chaos aus, und der Schmied muss kämpfen, um sein Haus zu verteidigen. Martial-Arts-Abenteuer unter der Regie von und mit RZA. Mit Russell Crowe und Lucy Liu. .. Aktion Abenteuer. The Man with the Iron Fists ', 'Ein Filmstar bereitet sich darauf vor, zum vierten Mal zu heiraten, eine Situation, die weder ihren Kindern noch ihrem Liebling ihrer drei Ex-Ehemänner gefällt. Bald kommt es zu Spannungen, als sich ihre Großfamilie für den großen Tag versammelt. Komödie mit Glenn Close, John Malkovich, Patrick Stewart und Minnie Driver. .. Komödie Romantik. The Wilde Wedding ', 'Ein Söldnerteam wird von der CIA beauftragt, nach einem Wissenschaftler zu suchen, der im Nahen Osten verschwunden ist. Mithilfe eines mysteriösen Priesters erkunden die Soldaten eine archäologische Ausgrabung, in der sie bald entdecken, dass eine schreckliche übernatürliche Kraft freigesetzt wurde. Horror mit Cuba Gooding Jr., Henry Rollins, Ron Perlman und Taryn Manning. .. Aktion Grusel. The Devil\'s Tomb ', 'Ein amerikanischer Militärbeamter untersucht den Tod einer Hubschrauberpilotin im Golfkrieg von 1990, um zu entscheiden, ob ihre Handlungen eine posthume Ehrenmedaille verdienen. In der Zwischenzeit versucht er, sich mit einem tragischen Vorfall auseinanderzusetzen, den er während des Konflikts selbst verursacht und vertuscht hat. Drama mit Denzel Washington, Meg Ryan, Lou Diamond Phillips, Scott Glenn und Matt Damon. .. Krieg Theater. Courage Under Fire ', 'Ein legendärer, aber pensionierter Killer wird zurück in seine tödlichen Wege gelockt, als sein Leben mit dem einer mysteriösen Frau kollidiert, die ein Geheimnis birgt. Jean Reno spielt die Hauptrolle. (2019) (91 Minuten). .. Aktion Thriller. Cold Blood Legacy ', 'Ester und ihre Mutter führen ein scheinbar bescheidenes B &amp; B - aber eines Nachts tauchen zwei unerwünschte Gäste auf und die Dinge nehmen eine gewalttätige Wendung. Gewalt / reife Themen / blinkende Bilder. (2019) (89 Minuten). .. Thriller Verbrechen. Coyote Lake ', 'Sa 07/03: Es wurden große Anstrengungen unternommen, um die Produktion in China anzukurbeln, nachdem das Coronavirus die Wirtschaft schwer getroffen hatte.. medizinisch Nachrichten Geschäft. China Battles To Boost Economy 07/03/20 ', 'Zu Ehren ihres verstorbenen Freundes begeben sich die Freunde aus Kindertagen, Seth Green, Dax Shepard und Matthew Lillard, auf die Spur der Schätze in die Wildnis. Komödie mit dem verstorbenen Burt Reynolds .... Abenteuer Komödie. Without a Paddle '"
'series_ep_tags',"'1990s', '19th century', '20 something', '20th century', 'action', 'addiction', 'adventure', 'adventurous', 'affairs', 'archive footage', 'arts', 'asia', 'assassin', 'ballet', 'behind the scenes', 'bio', 'bromance', 'business', 'china', 'classical music', 'combat', 'comedy', 'crime', 'criminal', 'dance', 'death & grief', 'demonic possession', 'documentary', 'drama', 'dysfunctional', 'eccentric', 'economy', 'ethics & morality', 'factual', 'falsely accused', 'family & relationship', 'fantasy', 'friendship', 'funny', 'global finance', 'gulf war', 'health', 'horror', 'infectious disease', 'investigation', 'latin america', 'loot & treasure', 'martial arts', 'medical', 'medicine conditions & illness', 'mental health', 'mexico', 'military',

In [17]:
infer_schema = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                         infer_feature_shape=True)
context.run(infer_schema)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/pipelinerlw8w3yc/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

---
#### **Example Validator**

The ExampleValidator component identifies anomalies in training and evaluation data by comparing data statistics computed by StatisticsGen component against a schema. The component can be configured to detect different classes of anomalies.

In [18]:
validate_stats = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=infer_schema.outputs['schema']
)
context.run(validate_stats)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/pipelinerlw8w3yc/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

In [19]:
train_anomalies_path = os.path.join(validate_stats.outputs['anomalies'].get()[0].uri, 'train/anomalies.pbtxt')
eval_anomalies_path = os.path.join(validate_stats.outputs['anomalies'].get()[0].uri, 'eval/anomalies.pbtxt')

train_anomalies = tfdv.load_anomalies_text(train_anomalies_path)
eval_anomalies = tfdv.load_anomalies_text(eval_anomalies_path)

tfdv.display_anomalies(train_anomalies)
tfdv.display_anomalies(eval_anomalies)

Here we didn't detect any anomalies since we didn't alter the schema to be more stringent. 

---

#### **Transform**

The transform component performas data transformations and feature engineering.

A major focus in ML Engineering is **consistent preprocessing** so as to avoid training-serving skew. Leveraging a library such as tensorflow transform allows us to build our preprocessing steps as TF graphs that can be deployed together with our model as a single artifact. This allows a requesting client to submit raw data and then the preprocessing happens on the deployed model graph.

In [20]:
%%writefile transform.py
"""
This file contains the preprocessing_fn callback function for the transform 
component in the tfx pipeline
"""
import tensorflow_transform as tft
import tensorflow as tf

_FEATURE_KEY = 'features'
_LABEL_KEY = 'series_ep_tags'
MAX_STRING_LENGTH = 277

# Filter out everything other than German characters 
regex_filter = '[^äÄöÖüÜßa-zA-Z]'

def _transformed_name(key):
    return key + '_xf'

def clean(text):
    """
    Clean up text input by removing everything but German words 

    Args: 
        text: tensors containing the synopsis. (batch_size/None, 1)

    Returns: 
        final: Cleaned text 
    """
    # Encoding needed to keep german characters
    lower = tf.strings.lower(text, 'utf-8')
    cleaned = tf.strings.regex_replace(lower, regex_filter, ' ')
    # Filter single letters
    single_letters_removed = tf.strings.regex_replace(cleaned, r'((^|\s)[äÄöÖüÜßa-zA-Z]{1})+\s', ' ')
    stripped = tf.strings.strip(single_letters_removed)
    # Replace multiple spaces with single space
    final = tf.strings.regex_replace(stripped, ' +', ' ')

    return final

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs 
    
    Args: 
        inputs: map from feature keys to raw not-yet-transformed-features 

    Returns: 
        Map from string feature key to transformed feature operations 
    """
    outputs = {}
    
    text = tf.squeeze(inputs[_FEATURE_KEY], axis=1)
    tags = inputs[_LABEL_KEY]

    cleaned = clean(text)
    text_tokens = tf.compat.v1.string_split(cleaned, ' ', result_type='RaggedTensor')
    text_tokens = text_tokens.to_tensor(shape=[None, MAX_STRING_LENGTH])
    text_indices = tft.compute_and_apply_vocabulary(
        text_tokens, vocab_filename='vocab', num_oov_buckets=1
    )
    
    # compute vocab of tags, do not apply due to serving issues
    _ = tft.vocabulary(tags, vocab_filename='tags')
    
    outputs[_transformed_name(_FEATURE_KEY)] = text_indices
    outputs[_LABEL_KEY] = tags
    
    return outputs



Overwriting transform.py


In [21]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    module_file='transform.py'
)
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/pipelinerlw8w3yc/Transform/transform_graph/5/.temp_path/tftransform_tmp/7c1ce4fb86b14270970b1ed937c9fd20/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/pipelinerlw8w3yc/Transform/transform_graph/5/.temp_path/tftransform_tmp/d4d70c5b725847c0a7ff62c5baa53210/assets
INFO:tensorflow:SavedModel written to: /tmp/pipelinerlw8w3yc/Transform/transform_graph/5/.temp_path/tftransform_tmp/d4d70c5b725847c0a7ff62c5baa53210/saved_model.pb
value: "\n\013\n\tConst_1:0\022\005vocab"

value: "\n\013\n\tConst_3:0\022\004tags"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
value: "\n\013\n\tConst_1:0\022\005vocab"

value: "\n\013\n\tConst_3:0\022\004tags"

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
value: "\n\013\n\tConst_1:0\022\005vocab"

value: "\n\013\n\tConst_3:0\022\004tags"

INFO:tensorflow:Saver not created 

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/tmp/pipelinerlw8w3yc/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "/tmp/pipelinerlw8w3yc/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/pipelinerlw8w3yc/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 14
        name: "TransformCache"
        )]
        ))

In [22]:
transform_dir = transform.outputs['transform_graph'].get()[0].uri
transformed_schema = tfdv.load_schema_text(os.path.join(transform_dir, 'transformed_metadata', 'schema.pbtxt'))

tfdv.display_schema(transformed_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'features_xf',INT,required,,"[0,304]"
'series_ep_tags',BYTES,,,-


We can easily inspect the outputs of the transform component. 

In [23]:
train_dataset_transformed = make_dataset(transform.outputs['transformed_examples'], 'train')
eval_dataset_transformed = make_dataset(transform.outputs['transformed_examples'], 'eval')

tf_transform_output = tft.TFTransformOutput(transform.outputs['transform_graph'].get()[0].uri)

VOCAB_FILE = tf_transform_output.vocabulary_file_by_name('vocab')
TAG_FILE = tf_transform_output.vocabulary_file_by_name('tags')

vocab_df = pd.read_csv(VOCAB_FILE, header=None)
tags_df = pd.read_csv(TAG_FILE, header=None)

for raw_record in train_dataset_transformed.take(1):
    print('\n {:=^80} \n'.format(" Decoded "))
    print(tf.io.parse_single_example(raw_record, tf_transform_output.transformed_feature_spec()))
    
display(vocab_df)
display(tags_df)


 =================================== Decoded ==================================== 

{'series_ep_tags': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fc1cee992d0>, 'features_xf': <tf.Tensor: shape=(277,), dtype=int64, numpy=
array([  4, 302, 158,  86,   9,  95,  44, 203,  14, 220,  10,  17,   6,
       245, 145,  18, 212,   7, 138, 247,  84,   3,   5,  52,  80,  42,
         8,   2,  15,  93,  74, 294,   9,  42,  57, 259, 185, 118,  77,
         0,  78, 211, 253,   2, 260,  72, 159, 128, 170, 257, 139, 120,
        38,   0, 162, 265, 183,  97, 269,  90, 238, 304, 304, 304, 304,
       304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304,
       304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304,
       304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304,
       304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304,
       304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304, 304,
       304, 304, 304, 304, 304, 

,0
0,und
1,die
2,mit
3,in
4,ein
...,...
299,anstrengungen
300,angriff
301,amp
302,amerikanischer


,0
0,thriller
1,drama
2,action
3,factual
4,usa
...,...
81,addiction
82,20th century
83,20 something
84,19th century


#### **Trainer**

The Trainer component is responsible for actually training the model. 

When running TFX on Kubeflow, we can kick out the training to AI platform, allowing for massive distributed training jobs with particular hardware specifications. 

As with Transform, we will need to provide a module file that defines our custom logic for this component. 

In [24]:
%%writefile model.py
import tensorflow as tf
import tensorflow_transform as tft
import pandas as pd
import numpy as np

from tensorflow.keras import callbacks, layers
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

from google.cloud import storage
import pickle

EMBEDDING_LOCATION = 'gs://ml-sandbox-101-tagging/data/processed/training_data/glove_data/glove_embedding_index.pkl'
MAX_STRING_LENGTH = 277


def create_tag_lookup_table(tag_file):
    table = tf.lookup.StaticVocabularyTable(
        tf.lookup.TextFileInitializer(
            tag_file,
            key_dtype=tf.string, key_index=tf.lookup.TextFileIndex.WHOLE_LINE,
            value_dtype=tf.int64, value_index=tf.lookup.TextFileIndex.LINE_NUMBER,
            delimiter=None),
        num_oov_buckets=1)
    return table


def label_transform(x, y, num_tags, table):
    """Use the number of classes to convert the sparse tag indicies to dense"""
    # Need to add one for out-of-vocabulary tags in eval dataset
    return (x, tf.cast(tf.sparse.to_indicator(table.lookup(y), vocab_size=num_tags + 1), tf.int32))

def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed fies"""
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def _input_fn(file_pattern, tf_transform_output, num_tags, table, batch_size=64, shuffle=True, epochs=None):
    """Generates features and label for tuning/training.
    Args:
        file_pattern: input tfrecord file pattern.
        tf_transform_output: A TFTransformOutput.
        batch_size: representing the number of consecutive elements of
          returned dataset to combine in a single batch
    Returns:
        A dataset that contains (features, indices) tuple where features
        is a dictionary of Tensors, and indices is a single Tensor of
        label indices.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        shuffle=shuffle,
        label_key='series_ep_tags',
        num_epochs=epochs
    )
    return dataset.map(lambda x, y: label_transform(x, y, num_tags, table))

class AutoTaggingModel:
    def __init__(
        self,
        embedding_file,
        embedding_dim,
        train_embedding,
        output_size,
        vocab_size,
        vocab_df,
        max_string_length,
    ):
        self.__embedding_file = embedding_file
        self.__embedding_dim = embedding_dim
        self.__vocab_size = vocab_size
        self.__vocab_df = vocab_df
        self.__train_embedding = train_embedding
        self.__output_size = output_size
        self.__max_string_length = max_string_length
        
        self.__initialize_embedding_matrix()
    
    def __initialize_embedding_matrix(self):
        storage_client = storage.Client()
        
        # Better way to do this with os.path?
        split_path = self.__embedding_file.split('/')
        bucket_name = split_path[2]
        blob_name = ('/').join(split_path[3:])
        
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        
        pickle_in = blob.download_as_string()
        file = pickle.loads(pickle_in)
        
        self.embedding_matrix = np.zeros((self.__vocab_size, 
                                     self.__embedding_dim))
        
        for i, word in enumerate(self.__vocab_df.values):
            embedding_vector = file.get(word[0])
            if embedding_vector is not None:
                self.embedding_matrix[i] = embedding_vector
        
    def embedding_layer(self):
        return layers.Embedding(
            input_dim=self.__vocab_size,
            output_dim=self.__embedding_dim,
            weights=[self.embedding_matrix],
            input_length=self.__max_string_length,
            trainable=self.__train_embedding,
        )

    def n_grams_channel(self, inputs, n_words_filter: int):
        channel = layers.Conv2D(256, kernel_size=(n_words_filter, self.__embedding_dim), activation="relu")(inputs)
        channel_mp = layers.MaxPool2D(pool_size=(channel.shape[1], 1))(channel)
        channel_final = layers.Flatten()(channel_mp)
        return channel_final
    
    def define_model(self):
        inputs = layers.Input(shape=(self.__max_string_length,), name='features')
        embedding = self.embedding_layer()(inputs) 
        channel_inputs = layers.Reshape(target_shape=(self.__max_string_length, self.__embedding_dim, 1))(embedding)
        channel1_final = self.n_grams_channel(channel_inputs, 3)
        channel2_final = self.n_grams_channel(channel_inputs, 4)
        channel3_final = self.n_grams_channel(channel_inputs, 5)
        channels_final = layers.Concatenate()(
            [channel1_final, channel2_final, channel3_final]
        )
        channels_final = layers.Dropout(rate=0.4)(channels_final)
        channels_final = layers.Dense(2000, "relu")(channels_final)
        predictions = layers.Dense(self.__output_size, "sigmoid")(channels_final)
        model = Model(inputs=inputs, outputs=predictions)

        return model

    def get_model(self):
        strategy = tf.distribute.MirroredStrategy()
        with strategy.scope():
            model = self.define_model()

            metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
            model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                loss=BinaryCrossentropy(),
                metrics=metrics,
            )
        return model

    
def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example."""

    model.tft_layer = tf_transform_output.transform_features_layer()
    
    tag_file = tf_transform_output.vocabulary_file_by_name('tags')
    tags = tf.io.read_file(tag_file)
    tags = tf.strings.strip(tags)
    tags = tf.strings.split(tags, '\n')

    @tf.function
    def serve_predict(features):
        """Returns the output to be used in the serving signature."""
        parsed_features = {
            'features': features,
        }

        transformed_features = model.tft_layer(parsed_features)
        outputs = model(transformed_features)
        
        return {"outputs": outputs,
               "tags": tags}

    return serve_predict


def run_fn(fn_args):
    """Train the model based on given args
    
    Args:
        fn_args: Holds args used to train the model as name/value pairs
    """
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    
    num_tags = tf_transform_output.vocabulary_size_by_name('tags')
    tag_file = tf_transform_output.vocabulary_file_by_name('tags')
    vocab_size = tf_transform_output.vocabulary_size_by_name('vocab')
    vocab_file = tf_transform_output.vocabulary_file_by_name('vocab')
    vocab_df = pd.read_csv(vocab_file, header=None)
    
    table = create_tag_lookup_table(tag_file)
    
    train_dataset = _input_fn(
        file_pattern=fn_args.train_files,
        tf_transform_output=tf_transform_output,
        num_tags=num_tags,
        table=table,
        batch_size=64)

    eval_dataset = _input_fn(
        file_pattern=fn_args.eval_files,
        tf_transform_output=tf_transform_output,
        batch_size=64,
        num_tags=num_tags,
        table=table)
    
    model = AutoTaggingModel(
        embedding_dim=300,
        train_embedding=True,
        embedding_file=EMBEDDING_LOCATION,
        output_size=num_tags + 1,
        vocab_size=vocab_size + 1,
        vocab_df=vocab_df,
        max_string_length=MAX_STRING_LENGTH).get_model()
    
    early_stopping_callback = callbacks.EarlyStopping(monitor='val_loss',
        min_delta=0.0001,
        patience=4,
        verbose=0,  
        mode='auto',  
        restore_best_weights=True)  

    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.1, 
        patience=2, 
        verbose=0, 
        mode='auto',
        min_delta=0.0001) 
    
    history = model.fit(
        train_dataset, 
        epochs=10,
        steps_per_epoch=fn_args.train_steps / 10, 
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[early_stopping_callback, reduce_lr]
    )
    
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(shape=[None, 1], dtype=tf.string, name="features")
        ),
    }
    
    model.save(
        fn_args.serving_model_dir, save_format="tf", signatures=signatures
    )




Overwriting model.py


In [25]:
_trainer_module_file = 'model.py'

In [26]:
trainer = Trainer(
    module_file=os.path.abspath(_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=infer_schema.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=10),
    eval_args=trainer_pb2.EvalArgs(num_steps=1))
context.run(trainer)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.6950 - precision: 0.1385 - recall: 0.56 - 2s 2s/step - loss: 0.6950 - precision: 0.1385 - recall: 0.5665 - val_loss: 0.6423 - val_precision: 0.0575 - val_recall: 0.0537
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6210 - precision: 0.3952 - recall: 0.20 - 0s 447ms/step - loss: 0.6210 - precision: 0.3952 - recall: 0.2014 - val_loss: 0.5726 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.5270 - precision: 0.4962 - recall: 0.08 - 0s 424ms/step - loss: 0.5270 - precision: 0.4962 - recall: 0.0888 - val_loss: 0.4756 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.4207 - precision: 0.6750 - recall: 0.07 - 0s 431ms/step - loss: 0.4207 - precision: 0.6750 - reca

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "/tmp/pipelinerlw8w3yc/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 16
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/tmp/pipelinerlw8w3yc/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 17
        name: "ModelRun"
        )]
        ))

---
#### **Validation**

After training the model, we want to validate its performance. This is where we can use the **TFMA** library. In some cases (including this one), we might also want to do more validation based on qualitative analysis on model behavior, as opposed to pure threshold-based validation. 

Here we demonstrate a custom component that runs the model on the entire validation set and provides a "blessing" if the evaluation metrics are above a certain threshold. 

With this particular model, we would perhaps want additional behavioral checks or even a human-in-the-loop to analyze the predictions in detail. There's a custom slack component that can be used for these purposes. 

In [27]:
from tfx.dsl.component.experimental.annotations import OutputDict
from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component

from tfx.utils import io_utils
from tfx.utils import path_utils

from tfx.types.standard_artifacts import Examples
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types import standard_artifacts

from tensorflow.keras.models import load_model
import model

In [28]:
@component
def ThresholdValidator(
    trained_model: InputArtifact[standard_artifacts.Model],
    transform_graph: InputArtifact[standard_artifacts.TransformGraph],
    transform_examples: InputArtifact[standard_artifacts.Examples],
    blessing: OutputArtifact[standard_artifacts.ModelBlessing],
    recall_threshold: Parameter[float] = 0.4,
    precision_threshold: Parameter[float] = 0.7, 
    ) -> OutputDict(precision=float, recall=float):
    '''Simple custom model validation component.'''
    
    loaded_model = load_model(os.path.join(trained_model.uri, 'serving_model_dir'))
    
    tf_transform_output = tft.TFTransformOutput(transform_graph.uri)
    num_tags = tf_transform_output.vocabulary_size_by_name('tags')
    tag_file = tf_transform_output.vocabulary_file_by_name('tags')
    
    eval_files = os.path.join(transform_examples.uri, 'eval/*')
    table = model.create_tag_lookup_table(tag_file)
    
    eval_dataset = model._input_fn(
        file_pattern=eval_files,
        tf_transform_output=tf_transform_output,
        batch_size=64,
        epochs=1,
        num_tags=num_tags,
        table=table,
        shuffle=False
    )
    results = loaded_model.evaluate(eval_dataset)
    
    if (results[1] >= precision_threshold) and (results[2] >= recall_threshold):
        io_utils.write_string_file(os.path.join(blessing.uri, 'BLESSED'), '')
        blessing.set_int_custom_property('blessed', 1)
    else:
        io_utils.write_string_file(os.path.join(blessing.uri, 'NOT_BLESSED'), '')
        blessing.set_int_custom_property('blessed', 0)
        

    return {
        'precision': results[1],
        'recall': results[2]
    }

First we can use a "strict" validator. 

In [29]:
validator_strict = ThresholdValidator(
    trained_model=trainer.outputs['model'],
    transform_graph=transform.outputs['transform_graph'],
    transform_examples=transform.outputs['transformed_examples'],
    recall_threshold=0.4,
    precision_threshold=0.7)
context.run(validator_strict)

1/1 [==============================]ss: 0.2815 - precision: 0.1818 - recall: 0.105 - 1s 610ms/step - loss: 0.2815 - precision: 0.1818 - recall: 0.1053


ExecutionResult(
    component_id: ThresholdValidator
    execution_id: 7
    outputs:
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 10
        type_id: 19
        uri: "/tmp/pipelinerlw8w3yc/ThresholdValidator/blessing/7"
        custom_properties {
          key: "blessed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ThresholdValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 19
        name: "ModelBlessing"
        )]
        )
        precision: Channel(
            type_name: Float
            artifacts: [Artifact(artifact: id: 11
        type_id: 20
        uri: "/tmp/pipelinerlw8w3yc/ThresholdValidator/precision/7/value"
        custom_properties {
          key: "name"
          value {
            string_value: "precision"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ThresholdValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 20
        name: "Float"
        )]
        )
        recall: Channel(
            type_name: Float
            artifacts: [Artifact(artifact: id: 12
        type_id: 20
        uri: "/tmp/pipelinerlw8w3yc/ThresholdValidator/recall/7/value"
        custom_properties {
          key: "name"
          value {
            string_value: "recall"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ThresholdValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 20
        name: "Float"
        )]
        ))

In [30]:
!ls {validator_strict.outputs['blessing'].get()[0].uri}

NOT_BLESSED


Looks like it didn't get blessed. Let's try with a less strict validator...

In [31]:
validator_lax = ThresholdValidator(
    trained_model=trainer.outputs['model'],
    transform_graph=transform.outputs['transform_graph'],
    transform_examples=transform.outputs['transformed_examples'],
    recall_threshold=0.0,
    precision_threshold=0.0)
context.run(validator_lax)

1/1 [==============================]ss: 0.2815 - precision: 0.1818 - recall: 0.105 - 0s 10ms/step - loss: 0.2815 - precision: 0.1818 - recall: 0.1053


ExecutionResult(
    component_id: ThresholdValidator
    execution_id: 8
    outputs:
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 13
        type_id: 19
        uri: "/tmp/pipelinerlw8w3yc/ThresholdValidator/blessing/8"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ThresholdValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 19
        name: "ModelBlessing"
        )]
        )
        precision: Channel(
            type_name: Float
            artifacts: [Artifact(artifact: id: 14
        type_id: 20
        uri: "/tmp/pipelinerlw8w3yc/ThresholdValidator/precision/8/value"
        custom_properties {
          key: "name"
          value {
            string_value: "precision"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ThresholdValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 20
        name: "Float"
        )]
        )
        recall: Channel(
            type_name: Float
            artifacts: [Artifact(artifact: id: 15
        type_id: 20
        uri: "/tmp/pipelinerlw8w3yc/ThresholdValidator/recall/8/value"
        custom_properties {
          key: "name"
          value {
            string_value: "recall"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ThresholdValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 20
        name: "Float"
        )]
        ))

In [32]:
!ls {validator_lax.outputs['blessing'].get()[0].uri}

BLESSED


Now we got the blessing! 

---

#### **Pusher**

The final component is the Pusher, where we actually push our validated model to a deployment target, in this case a bucket in GCS. We can then have tensorflow serving listening in on this bucket in order to pick up the latest model and serve the best predictions to customers/other downstream systems :)

In [33]:
!mkdir 'serving_test'

mkdir: cannot create directory ‘serving_test’: File exists


In [34]:
pusher = Pusher(
  model=trainer.outputs['model'],
  model_blessing=validator_strict.outputs['blessing'],
  push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_test')
  )
)
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 16
        type_id: 22
        uri: "/tmp/pipelinerlw8w3yc/Pusher/pushed_model/9"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 22
        name: "PushedModel"
        )]
        ))

In [35]:
!ls {pusher.outputs['pushed_model'].get()[0].uri}

When we use the output of our custom component it looks like the model wasn't actually pushed! 

In [36]:
pusher_lax = Pusher(
  model=trainer.outputs['model'],
  model_blessing=validator_lax.outputs['blessing'],
  push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_test')
  )
)
context.run(pusher_lax)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 17
        type_id: 22
        uri: "/tmp/pipelinerlw8w3yc/Pusher/pushed_model/10"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "serving_test/1604409715"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1604409715"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        , artifact_type: id: 22
        name: "PushedModel"
        )]
        ))

In [37]:
!ls {pusher_lax.outputs['pushed_model'].get()[0].uri}

assets	saved_model.pb	variables


In [38]:
display(read_only_store.get_artifacts_of_type_df('PushedModel'))

,URI,PUSHED,NAME,PRODUCER_COMPONENT,STATE,PUSHED_DESTINATION,PUSHED_VERSION
ID,,,,,,,
16,/tmp/pipelinerlw8w3yc/Pusher/pushed_model/9,,pushed_model,Pusher,published,-,-
17,/tmp/pipelinerlw8w3yc/Pusher/pushed_model/10,1,pushed_model,Pusher,published,serving_test/1604409715,1604409715


Here it was pushed. We see how TFX has built model validation into the pipeline so that only models of sufficiently high quality will be served.

---

#### **Running Inference**

Let's test our model!

In [39]:
timestamped_directory = os.listdir('serving_test')[-1]
os.environ['DIRECTORY'] = timestamped_directory

In [40]:
%%bash
saved_model_cli show \
    --dir serving_test/${DIRECTORY} \
    --tag_set serve \
    --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['features'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_features:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['outputs'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 87)
      name: StatefulPartitionedCall_1:0
  outputs['tags'] tensor_info:
      dtype: DT_STRING
      shape: (86)
      name: StatefulPartitionedCall_1:1
Method name is: tensorflow/serving/predict


Now we can feed in completely raw instances and the preprocessing will happen automatically, we don't need to worry about it at all. 

In [41]:
%%bash
saved_model_cli run \
    --dir serving_test/${DIRECTORY} \
    --tag_set serve \
    --signature_def serving_default \
    --outdir "results" \
    --overwrite \
    --input_expr 'features=[["Mysterythriller thriller, mystery, horror Thriller-Meister M. Night Shyamalan führt die Fäden aus Unbreakable und Split zusammen: Bruce Willis, Samuel L. Jackson und James McAvoy in einem faszinierenden Superhelden-Epos. "],["Aktuelle Themen aus Politik, Wirtschaft, Kultur, Sport, Gesellschaft und Wissenschaft aus dem In- und Ausland werden in ausführlichen Hintergrundberichten beleuchtet. In Stellungnahmen und Meinungsäußerungen wird auch unterschiedlichne Standpunkten Raum gegeben. "]]'

Result for output key outputs:
[[0.50923413 0.5073275  0.5056021  0.35554704 0.31516206 0.34388787
  0.28752083 0.36214107 0.3057989  0.30865192 0.30925357 0.35798198
  0.30739903 0.31910956 0.26497886 0.2699798  0.35773325 0.3185737
  0.2860229  0.35101676 0.15601262 0.12017858 0.15489668 0.11815253
  0.14898703 0.14549485 0.16113883 0.15878022 0.10980466 0.14486662
  0.12944716 0.12367603 0.13606659 0.13391149 0.14271337 0.13650417
  0.15427762 0.13833517 0.15064588 0.14701465 0.16292131 0.11394095
  0.12110546 0.14414886 0.12250456 0.13159618 0.12530905 0.14297819
  0.11980575 0.10352373 0.10865283 0.17013818 0.14194894 0.13357079
  0.10908404 0.1659551  0.11345819 0.14172384 0.12409824 0.16925675
  0.12363371 0.10419092 0.15049616 0.11633575 0.15108025 0.1575168
  0.14893287 0.1423097  0.11521956 0.16985428 0.12402827 0.12913907
  0.14495903 0.11709633 0.11854276 0.1196999  0.15417418 0.1455478
  0.1262596  0.11407328 0.13106579 0.12709501 0.15922564 0.15038136
  0.17092067 0.16067

2020-11-03 13:22:24.315256: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2300000000 Hz
2020-11-03 13:22:24.315699: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560c31f54d30 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-03 13:22:24.315743: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-11-03 13:22:24.315999: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from serving_test/1604409715/variables/variables


In [42]:
import numpy as np

In [43]:
predictions = np.load('results/outputs.npy')

In [44]:
sorted_predicted_tags = np.argsort(predictions, axis=1)[:, ::-1]

display(tags_df.iloc[sorted_predicted_tags[0][:10]])
display(tags_df.iloc[sorted_predicted_tags[1][:10]])

,0
0,thriller
1,drama
2,action
7,mystery
11,funny
16,comedy
3,factual
19,adventure
5,the far east
13,dysfunctional


,0
0,thriller
1,drama
2,action
3,factual
11,funny
16,comedy
7,mystery
19,adventure
5,the far east
13,dysfunctional


Let's clean up the files we created:

Alternatively, we can use the actual output of the model!

In [49]:
tags = np.load('results/tags.npy', allow_pickle=True)

In [59]:
print(tags[sorted_predicted_tags[0][:10]])
print(tags[sorted_predicted_tags[1][:10]])

[b'thriller' b'drama' b'action' b'mystery' b'funny' b'comedy' b'factual'
 b'adventure' b'the far east' b'dysfunctional']
[b'thriller' b'drama' b'action' b'factual' b'funny' b'comedy' b'mystery'
 b'adventure' b'the far east' b'dysfunctional']


We get the same results! 

**NOTE:** We will have one extra prediction for OOV tags (length of predictions is 87 and number of tags is 86). Not a problem if we only want the top few tags (as above), but can create a problem if we want to order all of the tags. Will need to either pad the tag list or disregard the final prediction. 

In [60]:
!gsutil cp -r serving_test/* gs://ml-sandbox-tagging-tfx-experiments/demo-model

Copying file://serving_test/1604399337/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://serving_test/1604399337/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://serving_test/1604399337/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://serving_test/1604399337/assets/tags [Content-Type=application/octet-stream]...
\ [4 files][ 31.5 MiB/ 31.5 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://serving_test/1604399337/assets/vocab [Content-Type=application/octet-stream]...
Copying file://serving_test/1604406070/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://serving_test/1604406070/variables/varia

In [61]:
%%bash
rm transform.py model.py
rm -rf serving_test
rm -rf results

---
---
## **C. Takeaways and Conclusion**

#### **TFX and Kubeflow Pipelines Comparison**

TFX can run on Kubeflow Pipelines, but the Kubeflow Pipelines SDK is also a way of defining an ML pipeline, and TFX libaries can be used within a pipeline defined using the KFP SDK. It can get confusing fairly quickly, so let's break down exactly what the relationship between these two technologies is.

**TFX**

Let's start with TFX: 


    "TFX is an integrated ML platform for developing and deploying production ML systems. A TFX pipeline is a sequence of components that implement an ML system. This TFX pipeline is designed for scalable, high-performance ML tasks. These tasks include modeling, training, validation, serving inference, and managing deployments. TFX pipelines are built around a few core libaries, including tensorflow data validation, tensorflow model analysis, and tensorflow transform"

We can think of TFX then as both a **collection of useful libraries** and as **a way of creating ML pipelines**. The libraries **do not** need to be used within the context of an end to end TFX pipeline; the libraries can be used in isolation or as part of a pipeline not defined using TFX components. 

As a way of defining pipelines, the TFX SDK is highly prescriptive: a basic TFX pipeline will include steps such as data validation, data transformation, model training, model validation and infrastructure validation. This opinionated way of creating pipelines reflects best practices defined by Google after years of experience productionizing ML pipelines. By using the TFX SDK to create ML pipelines, we will essentially end up following certain best pratices (e.g. data validation, model validation, consistent preprocessing between training/serving) without even really thinking about it, they essentially come "for free" simply by creating an end-to-end pipeline using the TFX SDK. A consequence of this is that many of our pipelines will end up looking quite similar, thus standardizing our workflows and *hopefully* streamlining the path to production. This does not mean that we sacrifice all flexibility, however, since we can also create custom components if existing TFX components don't have exactly the functionality we need. 


**Kubeflow Pipelines**

    Kubeflow is an open source Kubernetes framework for developing and running portable ML workloads. Kubeflow Pipelines is a Kubeflow service that lets you compose, orchestrate, and automate ML systems, where each component of the system can run on Kubeflow, Google Cloud, or other cloud platforms. The Kubeflow Pipelines platform consists of the following: 

     - A user interface for managing and tacking experiments, jobs, and runs
     - An engine for scheduling multistep ML workflows. KFP use Argo to orchestrate Kubernetes resources. 
     - A python SDK for defining and manipulating pipelines and components 
     - Notebooks for interacting with the system using the Python SDK.

Now the relationship should hopefully start to get a bit clearer. We can use the orchestration engine in Kubeflow Pipelines to schedule multistep ML workflows—including pipelines defined using the TFX SDK. We could also, however, define a multi-step ML workflow using **the Kubeflow Pipelines SDK**. Such a pipeline will consist of a set of containerized ML tasks, or components. Such components can be created using the following methods: 

 - Implementing a light-weight Python component
 - Creating a reusable component by creating a YAML file with a component specification describing the arguments to the component, the Docker container image URL to execute, and the outputs. 
 - Using predefined Google Cloud components that execute tasks on various managed services such as BigQuery, Dataflow, Dataproc and AI Platform. 
 
Since we have a lot of freedom in creating our own components when using the KFP SDK, there is nothing stopping us from using one of the TFX libraries as part of a component if we wanted to, for example, create our own data validation component that makes use of the tensorflow data validation library. However, we aren't limited to executing TFX-related services. A component in a kubeflow pipeline can execute any data or compute-related service, including Dataproc for SparkML jobs or AutoML. This gives us a lot of freedom to define highly customizable workflows. 

**Kubeflow Pipelines SDK vs TFX SDK**


|                     | TFX SDK                                       | KFP SDK                                  |Assessment        |
|-----------------    | ------------------------------------------- | ----------------------------------------|------------------|
|**Standardization**  | Very high                                   | Low                                     |**TFX**           |
|**Customizability**  | Medium                                      | Very high                               |**KFP**          |
|**Difficulty to Learn**   | High                                        | High                                    |**<-->**          |
|**Framework**        | Tensorflow                                  | Basically any                           |**KFP**           |
|**Metadata Integration**| Standardized and automatic                  | Manual and customizable                   |**<-->**        |

In short then, the decision whether to use the KFP or TFX SDKs to compose pipelines mostly rests on how flexible or standardized you want machine learning workflows to be. Of course, different teams within an organization will probably have conflicting viewpoints on this, with engineering teams favoring standardization and stability and data scientists favoring flexibility. 

One possible setup would be to start with Kubeflow Pipelines as a way to orchestrate and automate ML systems. For typical ML use cases and when possible, the team could write their pipelines using the TFX SDK and orchestrate using Kubeflow Pipelines. For more niche use-cases one could use the Kubeflow Pipelines SDK. That being said, if one chooses to use the KFP SDK to define the pipelines, I would highly recommend that the pipeline still follows the general pattern of a "typical" TFX pipeline, with custom components performing validation on the data, model, and infrastructure (perhaps even by using the TFX libraries). 


---
#### **When To Use TFX SDK to Define Pipelines**

 + If using TensorFlow and want to enforce best practices. 
 + Many pipelines are built for similar use-cases
 + Want standardized workflows
 
---
#### **When to Not Use TFX SDK to Define Pipelines**
 + Very early-stage experimentation (still good to be thinking ahead, though!)
 + Not using TensorFlow
 + When a lot of flexibility is **really needed** (is that flexibility really needed?)
     + "Research solutions that provide a small accuracy benefit at the cost of massive increases in system complexity are rarely wise practice"
     + "Flexibility does not come without costs. A more flexible system is one that is harder to get right in the first place as well as harder for us to maintain and to evolve."
 + If most of your pipelines and use cases are very different from each other.  

---
---

## **D. Resources**

 - [Hidden Technical Debt in Machine Learning Systems](https://papers.nips.cc/paper/5656-hidden-technical-debt-in-machine-learning-systems)
 - [Towards ML Engineering: A Brief History of TFX](https://arxiv.org/abs/2010.02013)
 - [Building Machine Learning Pipelines](https://www.oreilly.com/library/view/building-machine-learning/9781492053187/)
 - [TFX Documentation](https://www.tensorflow.org/tfx)
 - [Continuous Training for Production ML](https://www.usenix.org/system/files/opml19papers-baylor.pdf)
 
 ---
**KFP vs TFX**
 - [Spring ML Paper](https://springml.com/solution-overview/comparison-of-kubeflow-tfx/)
 - [Architecture for MLOps Blog Post](https://cloud.google.com/solutions/machine-learning/architecture-for-mlops-using-tfx-kubeflow-pipelines-and-cloud-build)
 - [Getting Started with Kubeflow Pipelines](https://cloud.google.com/blog/products/ai-machine-learning/getting-started-kubeflow-pipelines)
